## Simple Pressure Changer Example using Steam Property Package

This notebook demonstrates how to assemble, initialize, and run a simple unit model using pressure changer with steam property package. This breifly demonstrates the basics of setting up a unit model.  See the documentation for a more complete description of the framework.

## Imports
Standard Pyomo and IDAES core components are imported below along with Panda which will be used in this example to print a table of steam states in the unit model.

In [ ]:
import pyomo.environ as pe
from idaes.core import FlowsheetBlock, StateBlockBase
from idaes.unit_models.pressure_changer import PressureChanger
from idaes.property_models import iapws95_ph
import pandas as pd

## Create a Unit model

The lines below create a Pyomo concrete model, pressure changer unit model and steam property parameter block instance. The steam property parameter block contains constants that can be shared among all similar state blocks in a unit model. The iapws (international association of properties of water and steam) property model is used.

In [ ]:
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(default={"dynamic": False})
model.fs.properties = iapws95_ph.Iapws95ParameterBlock()

## Create a Pressure Changer Model

Here the pressure changer model is created using the defined property above. Four different types of pressure changer operations are considered;  
1. isothermal; constant temperature,
2. Adiabatic; net heat transfer of zero,
3. pump; assuming incompressible fluid,
4. isentropic; constant entropy.

We will test the isentropic assumption for this example.

In [ ]:
model.fs.pressure_changer = PressureChanger(default={"property_package": model.fs.properties,
                            "thermodynamic_assumption":'isentropic'})

## Fix Inlets and Initialize

In [ ]:
init_state = {
        "flow_mol":1e3,
        "pressure":1e6,
        "enth_mol":100000
    }
model.fs.pressure_changer.inlet[0].flow_mol.fix(1e3)
model.fs.pressure_changer.inlet[0].pressure.fix(1e6)
model.fs.pressure_changer.inlet[0].enth_mol.fix(100000)
model.fs.pressure_changer.deltaP.fix(500)
model.fs.pressure_changer.efficiency_isentropic.fix(0.9)

model.fs.pressure_changer.initialize(state_args=init_state)

## Solve the Disconnected Unit model

Solve the model after initialization.

In [ ]:
solver = pe.SolverFactory('ipopt') # Set up the solver IPOPT

In [ ]:
solver.solve(model, tee=True)

## Write a Function to Show States at Ports

Write a function that will give a nice tabulated overview of the conditions in the process.

In [ ]:
def state_table(m):
    head = ["State Block", 
        "Flow (mol/s)",
        "Temperature (K)",
        "Pressure (Pa)",
        "Enthalpy (J/mol)",
        "Vapor Fraction"]
    st = pd.DataFrame(columns=head)
    j = 0
    for c in model.component_objects():
        if isinstance(c, StateBlockBase):
            for i in c:
                row = [c[i].name, 
                       pe.value(c[i].flow_mol), 
                       pe.value(c[i].temperature), 
                       pe.value(c[i].pressure), 
                       pe.value(c[i].enth_mol),
                       pe.value(c[i].vapor_frac)]
                st.loc[j] = row
                j += 1
    return st

## Show States

The table below shows the properties of the state blocks created by the pressure changer unit model.

In [ ]:
state_table(model)